---
# Combinación de conjuntos de datos: fusionar y unir
---

Una característica esencial que ofrece Pandas son sus operaciones de unión y fusión en memoria de alto rendimiento.<br>
Si alguna vez ha trabajado con bases de datos, debería estar familiarizado con este tipo de interacción de datos. <br>
La interfaz principal para esto es la función ``pd.merge``, y veremos algunos ejemplos de cómo esto puede funcionar en la práctica.

Por conveniencia, comenzaremos redefiniendo la funcionalidad ``display()`` de la sección anterior , esto lo haremos al solo efecto de poder mostrar visualmente varios conjuntos de datos al mismo tiempo:

In [1]:
import pandas as pd
import numpy as np

class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)


# <span style="color:orange"> 1. Álgebra relacional <br>

El comportamiento implementado en ``pd.merge()`` es un subconjunto de lo que se conoce como *álgebra relacional*, que es un conjunto formal de reglas para manipular datos relacionales y forma la base conceptual de las operaciones disponibles en la mayoría de las bases de datos. <br>
La fortaleza del enfoque del álgebra relacional es que propone varias operaciones primitivas, que se convierten en los componentes básicos de operaciones más complicadas en cualquier conjunto de datos. <br>
Con este léxico de operaciones fundamentales implementado eficientemente en una base de datos u otro programa, se puede realizar una amplia gama de operaciones compuestas bastante complicadas.

Pandas implementa varios de estos bloques de construcción fundamentales en la función ``pd.merge()`` y el método relacionado ``join()`` de ``Series`` y ``Dataframe``.<br>
Como veremos, estos le permiten vincular de manera eficiente datos de diferentes fuentes.

## <span style="color:orange"> 2. Categorías de uniones </span>

La función ``pd.merge()`` implementa varios tipos de uniones: las uniones *uno a uno*, *muchos a uno* y *muchos a muchos*.<br>
Se accede a los tres tipos de combinaciones mediante una llamada idéntica a la interfaz ``pd.merge()``.<br> 
El tipo de unión realizada depende de la forma de los datos de entrada.<br>
Aquí mostraremos ejemplos simples de los tres tipos de fusiones y analizaremos las opciones detalladas más adelante.

### <span style="color:orange"> 2.1. Uniones uno a uno </span>

Quizás el tipo más simple de expresión de fusión es la unión uno a uno, que en muchos aspectos es muy similar a la concatenación de columnas.<br>
Como ejemplo concreto, considere los siguientes dos ``DataFrames`` que contienen información sobre varios empleados de una empresa:

In [2]:
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR']})
df1

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR


In [3]:
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
                    'hire_date': [2004, 2008, 2012, 2014]})
df2

,employee,hire_date
0,Lisa,2004
1,Bob,2008
2,Jake,2012
3,Sue,2014


In [4]:
display('df1', 'df2')

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
,employee,hire_date
0,Lisa,2004
1,Bob,2008
2,Jake,2012
3,Sue,2014


Para combinar esta información en un único ``DataFrame``, podemos usar la función ``pd.merge()``:

In [5]:
df3=pd.merge(df1,df2)
df3

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


La función ``pd.merge()`` reconoce que cada ``DataFrame`` tiene una columna de "employee" y se une automáticamente usando esta columna como clave.
El resultado de la fusión es un nuevo ``DataFrame`` que combina la información de las dos entradas.<br>
Tenga en cuenta que el orden de las entradas en cada columna no se mantiene necesariamente: en este caso, el orden de la columna "employee" difiere entre ``df1`` y ``df2``, y ``pd.merge()``La función tiene en cuenta esto correctamente. <br>
Además, tenga en cuenta que la fusión en general descarta el índice, excepto en el caso especial de fusiones por índice (consulte las palabras clave ``left_index`` y ``right_index``, que se analizan momentáneamente).

### <span style="color:orange"> 2.2. Join de muchos a uno
Las combinaciones de muchos a uno son combinaciones en las que una de las dos columnas clave contiene entradas duplicadas. <br>
Para el caso de muchos a uno, el ``DataFrame`` resultante conservará esas entradas duplicadas según corresponda.<br>
Considere el siguiente ejemplo de una unión de muchos a uno:

In [6]:
df4 = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR'],
                    'supervisor': ['Carly', 'Guido', 'Steve']})

df4


,group,supervisor
0,Accounting,Carly
1,Engineering,Guido
2,HR,Steve


In [7]:
display('df3','df4', 'pd.merge(df3,df4)')

df3
  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014

df4
         group supervisor
0   Accounting      Carly
1  Engineering      Guido
2           HR      Steve

pd.merge(df3,df4)
  employee        group  hire_date supervisor
0      Bob   Accounting       2008      Carly
1     Jake  Engineering       2012      Guido
2     Lisa  Engineering       2004      Guido
3      Sue           HR       2014      Steve

In [8]:
pd.merge(df3,df4)

,employee,group,hire_date,supervisor
0,Bob,Accounting,2008,Carly
1,Jake,Engineering,2012,Guido
2,Lisa,Engineering,2004,Guido
3,Sue,HR,2014,Steve


El ``DataFrame`` resultante tiene una columna adicional con la información del "supervisor", donde la información se repite en una o más ubicaciones según lo requieran las entradas.

### <span style="color:orange"> 2.3. Joins de muchos a muchos
Las uniones de muchos a muchos son un poco confusas conceptualmente, pero aun así están bien definidas. <br>
Si la columna clave en la matriz izquierda y derecha contiene duplicados, entonces el resultado es una combinación de muchos a muchos.<br>
Quizás esto quede más claro con un ejemplo concreto.<br>
Considere lo siguiente, donde tenemos un ``DataFrame`` que muestra una o más habilidades asociadas con un grupo en particular.<br>
Al realizar una unión de muchos a muchos, podemos recuperar las habilidades asociadas a cualquier persona individual:

In [9]:
df5 = pd.DataFrame({'group': ['Accounting', 'Accounting',
                              'Engineering', 'Engineering', 'HR', 'HR'],
                    'skills': ['math', 'spreadsheets', 'coding', 'linux',
                               'spreadsheets', 'organization']})

df5


,group,skills
0,Accounting,math
1,Accounting,spreadsheets
2,Engineering,coding
3,Engineering,linux
4,HR,spreadsheets
5,HR,organization


In [10]:
display('df1','df5', 'pd.merge(df1 , df5)')

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
,group,skills
0,Accounting,math
1,Accounting,spreadsheets
2,Engineering,coding
3,Engineering,linux
4,HR,spreadsheets


Estos tres tipos de uniones se pueden utilizar con otras herramientas de Pandas para implementar una amplia gama de funciones.<br>
Pero en la práctica, los conjuntos de datos rara vez son tan limpios como el que estamos trabajando aquí.<br>
En la siguiente sección consideraremos algunas de las opciones proporcionadas por ``pd.merge()`` que le permiten ajustar cómo funcionan las operaciones de unión.

## <span style="color:orange"> 3. Especificación de Merge Key
Ya hemos visto el comportamiento predeterminado de ``pd.merge()``: busca uno o más nombres de columnas coincidentes entre las dos entradas y lo usa como clave.<br>
Sin embargo, a menudo los nombres de las columnas no coinciden tan bien y ``pd.merge()`` proporciona una variedad de opciones para manejar esto.

### <span style="color:orange"> 3.1. La palabra clave ``on``

Lo más simple es que puedes especificar explícitamente el nombre de la columna clave usando la palabra clave ``on``, que toma un nombre de columna o una lista de nombres de columnas:

In [11]:
display('df1','df2', "pd.merge(df1 , df2 , on='employee')")

df1
  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR

df2
  employee  hire_date
0     Lisa       2004
1      Bob       2008
2     Jake       2012
3      Sue       2014

pd.merge(df1 , df2 , on='employee')
  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014

Esta opción sólo funciona si tanto el ``DataFrame`` izquierdo como el derecho tienen el nombre de columna especificado.

### <span style="color:orange"> 3.2. Las palabras clave ``left_on`` y ``right_on``

En ocasiones, es posible que desee fusionar dos conjuntos de datos con nombres de columnas diferentes; por ejemplo, es posible que tengamos un conjunto de datos en el que el nombre del empleado esté etiquetado como "nombre" en lugar de "empleado".<br>
En este caso, podemos usar las palabras clave ``left_on`` y ``right_on`` para especificar los nombres de las dos columnas:

In [12]:
df3 = pd.DataFrame({'name': ['Bob', 'Lisa','Jake', 'Sue'],
                    'salary': [70000, 80000, 120000, 90000]})

df3


,name,salary
0,Bob,70000
1,Lisa,80000
2,Jake,120000
3,Sue,90000


In [13]:
display('df1','df3', "pd.merge(df1, df3, left_on='employee', right_on='name')")

df1
  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR

df3
   name  salary
0   Bob   70000
1  Lisa   80000
2  Jake  120000
3   Sue   90000

pd.merge(df1, df3, left_on='employee', right_on='name')
  employee        group  name  salary
0      Bob   Accounting   Bob   70000
1     Jake  Engineering  Jake  120000
2     Lisa  Engineering  Lisa   80000
3      Sue           HR   Sue   90000

In [14]:
#display('df1','df3', "pd.merge(df1, df3)") # si no hay columnas en comun no hace el merge

El resultado tiene una columna redundante que podemos eliminar si lo deseamos, por ejemplo, usando el método ``drop()`` de ``DataFrame``s:

In [ ]:
pd.merge(df1, df3, left_on='employee', right_on='name').drop('name', axis=1) #axis=1 significa que quiero borrar 1 columna. axis=0 borra una fila

,employee,group,salary
0,Bob,Accounting,70000
1,Jake,Engineering,120000
2,Lisa,Engineering,80000
3,Sue,HR,90000


### <span style="color:orange"> 3.3. Las palabras clave ``left_index`` y ``right_index``

A veces, en lugar de fusionar en una columna, le gustaría fusionar en un índice.<br>
Por ejemplo, sus datos podrían verse así:

In [16]:
df1_set_index = df1.set_index('employee')
df1_set_index


,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR


In [17]:
df2_set_index = df2.set_index('employee')
df2_set_index

,hire_date
employee,
Lisa,2004
Bob,2008
Jake,2012
Sue,2014


In [18]:
df3_set_index= df3.set_index('name')
df3_set_index

,salary
name,
Bob,70000
Lisa,80000
Jake,120000
Sue,90000


Puedes usar el índice como clave para fusionar especificando los indicadores ``left_index`` y/o ``right_index`` en ``pd.merge()``:

In [19]:
display('df1_set_index', 'df2_set_index' , "pd.merge(df1_set_index,df2_set_index, left_index=True,right_index=True)")

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR
,hire_date
employee,
Lisa,2004
Bob,2008
Jake,2012


In [20]:
display('df1_set_index', 'df3_set_index' , "pd.merge(df1_set_index,df3_set_index, left_index=True,right_index=True)")

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR
,salary
name,
Bob,70000
Lisa,80000
Jake,120000


Para mayor comodidad, los ``DataFrame`` implementan el método ``join()``, que realiza una fusión que por defecto se une en índices:

In [21]:
display('df1_set_index','df2_set_index', 'df1_set_index.join(df2_set_index)')

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR
,hire_date
employee,
Lisa,2004
Bob,2008
Jake,2012


Si desea mezclar índices y columnas, puede combinar ``left_index`` con ``right_on`` o ``left_on`` con ``right_index`` para obtener el comportamiento deseado:

In [22]:
display('df1_set_index','df3'," pd.merge(df1_set_index ,df3, left_index=True, right_on='name')")

df1_set_index
                group
employee             
Bob        Accounting
Jake      Engineering
Lisa      Engineering
Sue                HR

df3
   name  salary
0   Bob   70000
1  Lisa   80000
2  Jake  120000
3   Sue   90000

 pd.merge(df1_set_index ,df3, left_index=True, right_on='name')
         group  name  salary
0   Accounting   Bob   70000
2  Engineering  Jake  120000
1  Engineering  Lisa   80000
3           HR   Sue   90000

## <span style="color:orange"> 4. Especificación de aritmética de conjuntos para joins

En todos los ejemplos anteriores hemos pasado por alto una consideración importante al realizar una unión: el tipo de aritmética de conjuntos utilizada en la unión.<br>
Esto aparece cuando aparece un valor en una columna clave pero no en la otra. Considere este ejemplo:


<div style="text-align: center;">
  <img src="Merges.png" width="400">
</div>


In [23]:
df6 = pd.DataFrame({'name': ['Peter', 'Paul', 'Mary'],
                    'food': ['fish', 'beans', 'bread']},
                   columns=['name', 'food'])

df6

,name,food
0,Peter,fish
1,Paul,beans
2,Mary,bread


In [24]:
df7 = pd.DataFrame({'name': ['Mary', 'Joseph'],
                    'drink': ['wine', 'beer']},
                   columns=['name', 'drink'])
df7

,name,drink
0,Mary,wine
1,Joseph,beer


In [25]:
display('df6','df7', "pd.merge(df6,df7)")

df6
    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

df7
     name drink
0    Mary  wine
1  Joseph  beer

pd.merge(df6,df7)
   name   food drink
0  Mary  bread  wine

Aquí hemos fusionado dos conjuntos de datos que tienen una sola entrada de "nombre" en común: María.<br>
De forma predeterminada, el resultado contiene la *intersección* de los dos conjuntos de entradas; esto es lo que se conoce como *inner join*.<br>
Podemos especificar esto explícitamente usando la palabra clave ``how``, que por defecto es ``"inner"``:

In [26]:
pd.merge(df6,df7,how='inner')

,name,food,drink
0,Mary,bread,wine


Otras opciones para la palabra clave  ``how`` keyword son ``'outer'``, ``'left'``, y ``'right'``.
Un *outer join* devuelve una unión sobre la unión de las columnas de entrada y completa todos los valores faltantes con NA:

### Veamos que pasa si aplicamos un metodo outer join

In [27]:
display('df6','df7', "pd.merge(df6 ,df7 , how= 'outer')")

df6
    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

df7
     name drink
0    Mary  wine
1  Joseph  beer

pd.merge(df6 ,df7 , how= 'outer')
     name   food drink
0  Joseph    NaN  beer
1    Mary  bread  wine
2    Paul  beans   NaN
3   Peter   fish   NaN

El *left join* y *right join* return une las entradas izquierda y derecha, respectivamente.<br>

### Veamos que pasa si aplicamos un metodo left join

In [28]:
display('df6','df7', "pd.merge(df6, df7 , how='left')")

df6
    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

df7
     name drink
0    Mary  wine
1  Joseph  beer

pd.merge(df6, df7 , how='left')
    name   food drink
0  Peter   fish   NaN
1   Paul  beans   NaN
2   Mary  bread  wine

Las filas de salida ahora corresponden a las entradas en la entrada izquierda. Usando ``how='right'`` funciona de manera similar.

Todas estas opciones se pueden aplicar directamente a cualquiera de los tipos de unión anteriores.

### Veamos que pasa si aplicamos un metodo right join

In [29]:
display('df6','df7', "pd.merge(df6, df7, how='right')")

df6
    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

df7
     name drink
0    Mary  wine
1  Joseph  beer

pd.merge(df6, df7, how='right')
     name   food drink
0    Mary  bread  wine
1  Joseph    NaN  beer

y si invertimos los df y vemos que pasa:

In [30]:
display('df6','df7', "pd.merge(df6, df7, how='left')")

df6
    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

df7
     name drink
0    Mary  wine
1  Joseph  beer

pd.merge(df6, df7, how='left')
    name   food drink
0  Peter   fish   NaN
1   Paul  beans   NaN
2   Mary  bread  wine

In [31]:
display('df6','df7', "pd.merge(df7, df6, how='left')")

df6
    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

df7
     name drink
0    Mary  wine
1  Joseph  beer

pd.merge(df7, df6, how='left')
     name drink   food
0    Mary  wine  bread
1  Joseph  beer    NaN

## <span style="color:orange"> 5. Nombres de columnas superpuestas: la palabra clave (Keyword) ``suffixes`` 
Finalmente, puede terminar en un caso en el que sus dos ``DataFrame``s de entrada tengan nombres de columnas en conflicto.<br>
Considere este ejemplo:

In [32]:
df8 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'rank': [1, 2, 3, 4]})
df8



,name,rank
0,Bob,1
1,Jake,2
2,Lisa,3
3,Sue,4


In [33]:
df9 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'rank': [3, 1, 4, 2]})
df9

,name,rank
0,Bob,3
1,Jake,1
2,Lisa,4
3,Sue,2


In [34]:
display('df8','df9', "pd.merge(df8,df9 ,on='name')")

df8
   name  rank
0   Bob     1
1  Jake     2
2  Lisa     3
3   Sue     4

df9
   name  rank
0   Bob     3
1  Jake     1
2  Lisa     4
3   Sue     2

pd.merge(df8,df9 ,on='name')
   name  rank_x  rank_y
0   Bob       1       3
1  Jake       2       1
2  Lisa       3       4
3   Sue       4       2

Debido a que la salida tendría dos nombres de columnas en conflicto, la función de combinación agrega automáticamente un sufijo ``_x`` o ``_y`` para que las columnas de salida sean únicas. <br>
Si estos valores predeterminados no son apropiados, es posible especificar un sufijo personalizado usando la palabra clave ``suffixes``:

In [35]:
display('df8','df9', 'pd.merge(df8 , df9 , on="name" , suffixes=["_L" , "_R"] )')

df8
   name  rank
0   Bob     1
1  Jake     2
2  Lisa     3
3   Sue     4

df9
   name  rank
0   Bob     3
1  Jake     1
2  Lisa     4
3   Sue     2

pd.merge(df8 , df9 , on="name" , suffixes=["_L" , "_R"] )
   name  rank_L  rank_R
0   Bob       1       3
1  Jake       2       1
2  Lisa       3       4
3   Sue       4       2

Estos sufijos funcionan en cualquiera de los patrones de unión posibles y también funcionan si hay varias columnas superpuestas.

## <span style="color:orange"> 6. Ejemplo: datos de los estados de EE. UU.

Las operaciones de fusión y unión surgen con mayor frecuencia cuando se combinan datos de diferentes fuentes.<br>
Aquí consideraremos un ejemplo de algunos datos sobre los estados de EE. UU. y sus poblaciones.<br>
Los archivos de datos se pueden encontrar en http://github.com/jakevdp/data-USstates/:

In [36]:
# Obtenemos mediante url las direcciones de los dataset:
url1 = 'https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-population.csv'
url2 = 'https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-areas.csv'
url3 = 'https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-abbrevs.csv'

Echemos un vistazo a los tres conjuntos de datos, usando la función ``read_csv()`` de Pandas:

In [39]:
pop = pd.read_csv(url1)
areas = pd.read_csv(url2)
abbrevs = pd.read_csv(url3)

display('pop.sample(5)','areas.sample(5)','abbrevs.sample(5)')

pop.sample(5)
     state/region     ages  year  population
2177           VT  under18  1997    150040.0
2253           VA  under18  1991   1548258.0
2507          USA  under18  1996  70233512.0
126            AZ    total  1998   4883342.0
2147           UT    total  1996   2067976.0

areas.sample(5)
          state  area (sq. mi)
14         Iowa          56276
7      Delaware           1954
51  Puerto Rico           3515
44      Vermont           9615
3      Arkansas          53182

abbrevs.sample(5)
             state abbreviation
23   New Hampshire           NH
40  South Carolina           SC
18       Louisiana           LA
2          Arizona           AZ
15            Iowa           IA

Dada esta información, supongamos que queremos calcular un resultado relativamente sencillo: clasificar los estados y territorios de EE. UU. según su densidad de población en 2010.<br>
Claramente tenemos los datos aquí para encontrar este resultado, pero tendremos que combinar los conjuntos de datos para encontrar el resultado.

Comenzaremos con una fusión de muchos a uno que nos dará el nombre completo del estado dentro de la población ``DataFrame``.<br>
Queremos fusionarnos en función de la columna ``estado/región`` de ``pop`` y la columna ``abreviatura`` de ``abbrevs``.<br>
Usaremos ``how='outer'`` para asegurarnos de que no se desperdicie ningún dato debido a etiquetas que no coinciden.

In [41]:
# Merge pop con abbrevs basado en la columna 'state/region' en pop y 'abbreviation' en abbrevs. Fusion de muchos a 1 
merge = pd.merge(pop , abbrevs , how='outer', left_on='state/region', right_on='abbreviation')
merge

,state/region,ages,year,population,state,abbreviation
0,AK,total,1990,553290.0,Alaska,AK
1,AK,under18,1990,177502.0,Alaska,AK
2,AK,total,1992,588736.0,Alaska,AK
3,AK,under18,1991,182180.0,Alaska,AK
4,AK,under18,1992,184878.0,Alaska,AK
...,...,...,...,...,...,...
2539,WY,under18,1993,137458.0,Wyoming,WY
2540,WY,total,1991,459260.0,Wyoming,WY
2541,WY,under18,1991,136720.0,Wyoming,WY
2542,WY,under18,1990,136078.0,Wyoming,WY


In [ ]:
# Eliminar la columna 'abbreviation' ya que es redundante
merge = merge.drop('abbreviation', axis=1)



KeyError: "['abbreviation'] not found in axis"

In [44]:
merge.head()

,state/region,ages,year,population,state
0,AK,total,1990,553290.0,Alaska
1,AK,under18,1990,177502.0,Alaska
2,AK,total,1992,588736.0,Alaska
3,AK,under18,1991,182180.0,Alaska
4,AK,under18,1992,184878.0,Alaska


Verifiquemos nuevamente si hubo discrepancias aquí, lo cual podemos hacer buscando filas con valores nulos:

In [45]:
merge.isnull().sum()

state/region     0
ages             0
year             0
population      20
state           96
dtype: int64

Parte de la información de la ``población`` es nula; ¡Averigüemos cuáles son!

In [46]:
merge[merge['population'].isnull()]

,state/region,ages,year,population,state
1872,PR,under18,1990,NaN,NaN
1873,PR,total,1990,NaN,NaN
1874,PR,total,1991,NaN,NaN
1875,PR,under18,1991,NaN,NaN
1876,PR,total,1993,NaN,NaN
1877,PR,under18,1993,NaN,NaN
1878,PR,under18,1992,NaN,NaN
1879,PR,total,1992,NaN,NaN
1880,PR,under18,1994,NaN,NaN
1881,PR,total,1994,NaN,NaN


Parece que todos los valores nulos de población son de Puerto Rico anteriores al año 2000; Es probable que esto se deba a que estos datos no están disponibles en la fuente original.

Más importante aún, vemos también que algunas de las nuevas entradas de ``state`` también son nulas, lo que significa que no había ninguna entrada correspondiente en la clave ``abrevs``.<br>
Averigüemos qué regiones carecen de esta coincidencia:

In [48]:
merge.loc[merge['state'].isnull() , 'state/region'].unique()

array(['PR', 'USA'], dtype=object)

Podemos inferir rápidamente el problema: nuestros datos de población incluyen entradas para Puerto Rico (PR) y los Estados Unidos en su conjunto (EE.UU.), mientras que estas entradas no aparecen en la clave de abreviatura estatal.<br>
Podemos solucionarlos rápidamente completando las entradas apropiadas:

In [52]:
merge.loc[merge['state/region'] == 'PR', 'state'] = 'Puerto Rico'
merge.loc[merge['state/region'] == 'USA', 'state'] = 'United States'
merge.isnull().any()

state/region    False
ages            False
year            False
population       True
state           False
dtype: bool

In [53]:
merge.isnull().sum()

state/region     0
ages             0
year             0
population      20
state            0
dtype: int64

No más valores nulos en la columna ``estado``: ¡ya estamos listos!

Ahora podemos fusionar el resultado con los datos del área usando un procedimiento similar.<br>
Al examinar nuestros resultados, querremos unirnos en la columna ``estado`` en ambos:

Nuevamente, revisemos los valores nulos para ver si hubo alguna discrepancia:

In [55]:
final = pd.merge(merge , areas , on='state', how='left')
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AK,total,1990,553290.0,Alaska,656425.0
1,AK,under18,1990,177502.0,Alaska,656425.0
2,AK,total,1992,588736.0,Alaska,656425.0
3,AK,under18,1991,182180.0,Alaska,656425.0
4,AK,under18,1992,184878.0,Alaska,656425.0


Hay valores nulos en la columna ``área``; Podemos echar un vistazo para ver qué regiones se ignoraron aquí:

In [57]:
final.isnull().sum()

state/region      0
ages              0
year              0
population       20
state             0
area (sq. mi)    48
dtype: int64

Vemos que nuestras ``áreas`` ``DataFrame`` no contiene el área de Estados Unidos en su conjunto.<br>
Podríamos insertar el valor apropiado (usando la suma de todas las áreas estatales, por ejemplo), pero en este caso simplemente eliminaremos los valores nulos porque la densidad de población de todo Estados Unidos no es relevante para nuestra discusión actual:

In [59]:
final.dropna(inplace=True) #nos quita los valores donde encuentra na
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AK,total,1990,553290.0,Alaska,656425.0
1,AK,under18,1990,177502.0,Alaska,656425.0
2,AK,total,1992,588736.0,Alaska,656425.0
3,AK,under18,1991,182180.0,Alaska,656425.0
4,AK,under18,1992,184878.0,Alaska,656425.0


In [60]:
final.isnull().sum()

state/region     0
ages             0
year             0
population       0
state            0
area (sq. mi)    0
dtype: int64

Ahora tenemos todos los datos que necesitamos. Para responder a la pregunta de interés, seleccionemos primero la parte de los datos correspondiente al año 2010 y la población total.

In [ ]:
data_2010 = final[(final['year']==2010) & (final['ages']=='total')] #con esto genero un subset
data_2010.sample(10)


,state/region,ages,year,population,state,area (sq. mi)
2501,WY,total,2010,564222.0,Wyoming,97818.0
965,MD,total,2010,5787193.0,Maryland,12407.0
1722,OH,total,2010,11545435.0,Ohio,44828.0
1253,MT,total,2010,990527.0,Montana,147046.0
1194,MO,total,2010,5996063.0,Missouri,69709.0
2346,VT,total,2010,625793.0,Vermont,9615.0
1973,SC,total,2010,4636361.0,South Carolina,32007.0
197,CA,total,2010,37333601.0,California,163707.0
1578,NM,total,2010,2064982.0,New Mexico,121593.0
283,CO,total,2010,5048196.0,Colorado,104100.0


Ahora veamos los Estados con una poblacion mayor a 5 Millones

In [64]:
final_limpio = final[(final['ages']=='total')] 
final_limpio.sample(5)

,state/region,ages,year,population,state,area (sq. mi)
486,GA,total,2013,9992167.0,Georgia,59441.0
2255,UT,total,1990,1731223.0,Utah,84904.0
264,CO,total,2001,4425687.0,Colorado,104100.0
2157,TX,total,2011,25640909.0,Texas,268601.0
2064,TN,total,2012,6454914.0,Tennessee,42146.0


In [65]:
estados_grandes = final_limpio[final_limpio['population']>5000000]
estados_grandes

,state/region,ages,year,population,state,area (sq. mi)
145,AZ,total,2012,6551149.0,Arizona,114006.0
147,AZ,total,2011,6468796.0,Arizona,114006.0
149,AZ,total,2010,6408790.0,Arizona,114006.0
151,AZ,total,2013,6626624.0,Arizona,114006.0
152,AZ,total,2009,6343154.0,Arizona,114006.0
...,...,...,...,...,...,...
2439,WI,total,2013,5742713.0,Wisconsin,65503.0
2440,WI,total,2009,5669264.0,Wisconsin,65503.0
2442,WI,total,2010,5689060.0,Wisconsin,65503.0
2445,WI,total,2011,5708785.0,Wisconsin,65503.0


In [67]:
estados_grandes['state'].unique()

array(['Arizona', 'California', 'Colorado', 'Florida', 'Georgia',
       'Illinois', 'Indiana', 'Massachusetts', 'Maryland', 'Michigan',
       'Minnesota', 'Missouri', 'North Carolina', 'New Jersey',
       'New York', 'Ohio', 'Pennsylvania', 'Tennessee', 'Texas',
       'Virginia', 'Washington', 'Wisconsin'], dtype=object)